In [1]:
import numpy as np

In [2]:
np.random.seed(709)
x = np.random.randint(-5, 10, (5, 3))
print(f'原始矩阵{x.shape}：')
print(x)
print('='*20)

X = x.T
print(f'原始矩阵的转置矩阵{X.shape}：')
print(X)
print('='*20)

mu = X.mean(axis=1)[:, None]
print(f'转置矩阵的按行平均值{mu.shape}：')
print(mu)
print('='*20)

A = X - mu
print(f'转置矩阵的按行进行均值零化{A.shape}：')
print(A)
print('='*20)

C = (A @ A.T) / x.shape[0]
print(f'协方差矩阵{C.shape}：')
print(C)
print('='*20)

D, V = np.linalg.eig(C)
idx = D.argsort()[::-1]
D = D[idx]
V = V[:, idx]
print(f'协方差矩阵的特征值：{D}')
print(f'协方差矩阵的特征向量{V.shape}：')
print(V)
print('='*20)

P = V.T
print(f'P is {P.shape}：')
print(P)
print('='*20)

print(f'协方差对角矩阵 = P @ C @ P.T')
print(P @ C @ P.T)
print('='*20)

原始矩阵(5, 3)：
[[ 2  8  6]
 [ 7 -5  6]
 [ 1 -1  0]
 [-1 -5 -5]
 [ 5 -4  8]]
原始矩阵的转置矩阵(3, 5)：
[[ 2  7  1 -1  5]
 [ 8 -5 -1 -5 -4]
 [ 6  6  0 -5  8]]
转置矩阵的按行平均值(3, 1)：
[[ 2.8]
 [-1.4]
 [ 3. ]]
转置矩阵的按行进行均值零化(3, 5)：
[[-0.8  4.2 -1.8 -3.8  2.2]
 [ 9.4 -3.6  0.4 -3.6 -2.6]
 [ 3.   3.  -3.  -8.   5. ]]
协方差矩阵(3, 3)：
[[ 8.16 -3.08 11.4 ]
 [-3.08 24.24  6.4 ]
 [11.4   6.4  23.2 ]]
协方差矩阵的特征值：[31.96814626 23.04137495  0.59047879]
协方差矩阵的特征向量(3, 3)：
[[ 0.30933008  0.45103018  0.83718975]
 [ 0.53038    -0.81254911  0.24178711]
 [ 0.78931107  0.36923667 -0.49056327]]
P is (3, 3)：
[[ 0.30933008  0.53038     0.78931107]
 [ 0.45103018 -0.81254911  0.36923667]
 [ 0.83718975  0.24178711 -0.49056327]]
协方差对角矩阵 = P @ C @ P.T
[[ 3.19681463e+01  1.50428271e-14 -9.41172213e-15]
 [ 1.45019850e-14  2.30413749e+01 -1.67037218e-15]
 [-8.97555714e-15 -2.01348774e-15  5.90478793e-01]]


In [3]:
# 降维验证
# 将3维原数据降到2维
x_r = P[0:2, :] @ x.T
print(f'原始的3维数据，每行代表一个维度，每列代表一条记录')
print(x.T)
print('='*20)

print(f'降到2维的数据，每行代表一个维度，每列代表一条记录')
print(x_r)
print('='*20)

print(f'原数据与降维后的数据模长比较，绝对值越小表示降维所带来的信息丢失越少')
print(np.linalg.norm(x.T, axis=0) - np.linalg.norm(x_r, axis=0))
print('='*20)

原始的3维数据，每行代表一个维度，每列代表一条记录
[[ 2  7  1 -1  5]
 [ 8 -5 -1 -5 -4]
 [ 6  6  0 -5  8]]
降到2维的数据，每行代表一个维度，每列代表一条记录
[[ 9.59756654  4.24927698 -0.22104992 -6.9077854   5.73961895]
 [-3.38291248  9.43537686  1.26357929  1.76553201  8.45924072]]
原数据与降维后的数据模长比较，绝对值越小表示降维所带来的信息丢失越少
[0.02172436 0.14001179 0.1314448  0.01158955 0.02432981]
